# Day 17

https://adventofcode.com/2022/day/17

Used these 2 sources to find solution. Part II was much harder. Hyper-Neutrino used a maybe more elegant way for rock movement and collision detection by using imaginary numbers. Otherwise both solutions are very similar. 

https://github.com/jonathanpaulson/AdventOfCode/blob/master/2022/17.py

https://github.com/hyper-neutrino/advent-of-code/blob/main/2022/day17p2.py

## Part 1

In [1]:
test_data = """\
>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"""
test_data

'>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

In [2]:
def get_shape(idx, y):
    idx = idx % 5
    # ..++++.
    if idx == 0: 
        return [(i+2, y) for i in range(4)]
    # ...+...
    # ..+++..
    # ...+...
    elif idx == 1: 
        return [(3, y+2), (3, y)] + [(i+2, y+1) for i in range(3)]
    # ....+..
    # ....+..
    # ..+++..
    elif idx == 2:
        return [(4, y+2), (4, y+1)] + [(i+2, y) for i in range(3)]
    # ..+....
    # ..+....
    # ..+....
    # ..+....
    elif idx == 3:
        return [(2, y+i) for i in range(4)]
    # ..++...
    # ..++...
    elif idx == 4:
        return [(2+i, y+j) for i in range(2) for j in range(2)]
    assert False

def move_left(shape):
    if any([_x[0] <= 0 for _x in shape]):
        return shape
    else:
        return set([(x-1, y) for x, y in shape])

def move_right(shape):
    if any([_x[0] >= 6 for _x in shape]):
        return shape
    else:
        return set([(x+1, y) for x, y in shape])

def move_down(shape):
    return set([(x, y-1) for x, y in shape])

def move_up(shape):
    return set([(x, y+1) for x, y in shape])


def solution1(data, N=2022):
    space = set([(x, 0) for x in range(7)])
    top = 0
    jet_idx = 0
    for i in range(N):
        s = set(get_shape(i, top+4))
        #print("new shape:", s)
        while True:
            jet = data[jet_idx]
            jet_idx = (jet_idx + 1) % len(data)
            if jet == "<":
                s = move_left(s)
                if s & space: # check for collision 
                    s = move_right(s) # undo
            else:
                s = move_right(s)
                if s & space:
                    s = move_left(s)
            s = move_down(s)
            if s & space:
                space |= move_up(s)
                top = max([_x[1] for _x in space])
                break
    print("Solution:", top)
    return top

sol1 = solution1(test_data)
assert sol1 == 3068
print("test passed")

Solution: 3068
test passed


In [3]:
with open("day17.txt") as f:
    inp_data = f.read()

sol1 = solution1(inp_data)
assert sol1 == 3163
print("test passed")

Solution: 3163
test passed


## Part 2

In [4]:
print("Part 2")

def signature(R):
    """Create a signature (x, y) of upper 30 rows, with height reference to top rock"""
    maxY = max([y for (x,y) in R])
    return frozenset([(x, maxY - y) for (x,y) in R if maxY - y <= 30])


def solution2(data):
    N = 1000000000000

    # dict of seen patterns
    SEEN = {}
    space = set([(x, 0) for x in range(7)])
    top = 0  # keep track of highest rock part
    i = 0   # nr of rocks
    jet_idx = 0   # index of jet action
    added = 0  # added height by repeating pattern
    while i < N:
        # emit rock
        s = set(get_shape(i, top+4))

        while True:
            jet = data[jet_idx]
            jet_idx = (jet_idx + 1) % len(data)
            if jet == "<":
                s = move_left(s)
                if s & space: # check for collision 
                    s = move_right(s) # undo
            else:
                s = move_right(s)
                if s & space:
                    s = move_left(s)
            s = move_down(s)
            if s & space:
                space |= move_up(s)
                top = max([y for (x, y) in space])
                
                # key for pattern recognition: (jet index, type of rock, x-y-coordinates of occupied 30 upper rows)
                key = (jet_idx, i%5, signature(space))

                if key in SEEN and i >= 2022:
                    # repeated pattern detected, get old nr. of rocks and old height
                    (i_old, top_old) = SEEN[key]
                    # height of pattern
                    dy = top - top_old
                    # number of rocks in pattern
                    di = i - i_old

                    # amount of repetitions of pattern to get just under limit N
                    amt = (N-i) // di
                    # added height by repeated patterns
                    added += amt*dy
                    # increase nr. of rock by repetition x nr. of rocks in pattern
                    i += amt*di
                    assert i <= N
                    # clear seen dict
                    SEEN = {}

                SEEN[key] = (i, top)

                break
        i += 1
    sol2 = top + added
    print("added by pattern:", added)
    print("Solution:", top + added)
    return sol2

sol2 = solution2(test_data)
assert sol2 == 1514285714288
print("test passed")

Part 2
added by pattern: 1514285711189
Solution: 1514285714288
test passed


In [5]:
sol2 = solution2(inp_data)
assert sol2 == 1560932944615
print("test passed")

added by pattern: 1560932939229
Solution: 1560932944615
test passed
